<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [43]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def calcular_TVD(dict_d1, dict_d2):
    """Distancia Total de Variación (TVD) entre dos distribuciones."""
    soporte_total = set(dict_d1) | set(dict_d2)
    return 0.5 * sum(abs(dict_d1.get(k, 0) - dict_d2.get(k, 0)) for k in soporte_total)

def calcular_momentos(dict_fpm):
    """Calcula esperanza, varianza y desviación estándar de una distribución discreta."""
    esperanza = sum(x * p for x, p in dict_fpm.items())
    varianza = sum((x - esperanza)**2 * p for x, p in dict_fpm.items())
    return esperanza, varianza, varianza**0.5

# --------------------------- Pandas ---------------------------

def generar_df_distribucion(dict_fpm):
    """Genera un DataFrame a partir de un diccionario FPM cualquiera."""
    soporte = list(dict_fpm.keys())
    data = [{
        "Datos": f"P({x})",
        "Valor": dict_fpm.get(x, 0)
    } for x in soporte]

    esp, var, desv = calcular_momentos(dict_fpm)

    resumen = [
        {"Datos": "Esperanza", "Valor": esp},
        {"Datos": "Varianza", "Valor": var},
        {"Datos": "Desv. Est.", "Valor": desv},
    ]

    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def comparar_df_distribuciones(dict_fpm_A, dict_fpm_B):
    df_A = generar_df_distribucion(dict_fpm_A)
    df_B = generar_df_distribucion(dict_fpm_B)

    df_comparacion = pd.DataFrame({
        "Datos": df_B["Datos"],
        "A": df_A["Valor"],
        "B": df_B["Valor"],
    })
    df_comparacion["Diferencia"] = (df_comparacion["A"] - df_comparacion["B"]).abs()

    return df_comparacion

# --------------------------- Calificadores / Analizadores ---------------------------
def calificar_generador_de_muestras(generador, dict_fpm_real, n=100_000):
  soporte = list(dict_fpm_real.keys())

  muestras, tiempos = zip(*(medir_tiempo(generador) for _ in range(n)))
  frec = Counter(muestras)
  dict_fpm_estimada = {x: frec.get(x, 0) / n for x in soporte}

  df_comparacion = comparar_df_distribuciones(dict_fpm_estimada, dict_fpm_real)
  tvd = calcular_TVD(dict_fpm_estimada, dict_fpm_real)
  t_prom = np.mean(tiempos)

  print("Estimada vs Real")
  print(df_comparacion.to_string(index=False))
  print()
  print(f"Distancia total de variación: {tvd:.6f}")
  print(f"Tiempo promedio de ejecución: {t_prom} segundos")

def analizar_generador_de_muestras(generador, n=100_000):
  muestras, tiempos = zip(*(medir_tiempo(generador) for _ in range(n)))

  frec = Counter(muestras)
  soporte = sorted(list(frec.keys()))
  dict_fpm_estimada = {x: frec.get(x, 0) / n for x in soporte}
  df_fpm_estimada = generar_df_distribucion(dict_fpm_estimada)

  t_prom = np.mean(tiempos)

  print("FPM estimada:")
  print(df_fpm_estimada.to_string(index=False))
  print()
  print(f"Tiempo promedio de ejecución: {t_prom} segundos")

# --------------------------- Utilidades ---------------------------

def ordenar_dict_fpm_por_probabilidad(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  dict_fda = {}
  acc = 0
  for v, p in dict_fpm.items():
    acc += p
    dict_fda[v] = acc
  return dict_fda

# def TI(fpm): # soporte subarreglo de [0, 1, ..., inf]
#   u = random.random()
#   i = 0; i_fda = fpm(0)
#   while u >= i_fda:
#     i += 1; i_fda += fpm(i)
#   return i

def TI(dict_fda):
  u = random.random()
  for v, p in dict_fda.items():
    if u < p:
      return v

# def crear_dict_fpm(soporte, fpm):
#   return {v: fpm(v) for v in soporte}

# dict_fpm = dict(gen_fpm)

In [68]:
# Estime P(Y > 2) con λ = 10, y 1000 repeticiones para la variable Poisson,
# simulando con método de transformada inversa común e inversa mejorado.

LAMBDA = 10

def gen_poisson(lambd):
  """Generador de probabilidades de la distribución Poisson de parámetro lambda."""
  p_anterior = math.exp(-lambd)  # P(X=0)
  acc = p_anterior
  yield 0, p_anterior
  i = 1
  while acc < 1 - 1e-10:  # Debido a los redondeos de flotantes
    p_actual = (lambd / i) * p_anterior  # P(X=i) = (lambda/i) * P(X=i-1)
    yield i, p_actual
    acc += p_actual
    p_anterior = p_actual
    i += 1

dict_fpm = dict(gen_poisson(LAMBDA))

dict_fda = calcular_dict_fda(dict_fpm)
TI_sin_orden = lambda: TI(dict_fda)

dict_fda_ordenado = calcular_dict_fda(ordenar_dict_fpm_por_probabilidad(dict_fpm))
TI_ordenado = lambda: TI(dict_fda_ordenado)

def probabilidad(funcion_booleana, n):
    prob = sum(funcion_booleana() for _ in range(n))
    return prob / n

def Y_mayor_a_2(generador_de_muestras):
  return generador_de_muestras() > 2

n = 10_000
print(probabilidad(lambda: Y_mayor_a_2(TI_sin_orden), n))
print(probabilidad(lambda: Y_mayor_a_2(TI_ordenado), n))

print(1 - dict_fda[2]) # mejor aproximacion (solo erra en flotantes bajos)

0.9963
0.9982
0.9972306042844884
